In [1]:
import os
import time
import argparse
import random
# from abc import ABC

import cv2
import numpy as np
import segmentation_models_pytorch as smp
import torch
# import torch.nn.functional as F
from torch.utils.data import DataLoader
from tqdm import tqdm
from PIL import Image
from matplotlib import cm

from scipy.ndimage import label
import numpy as np

import lightning as L


import os
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

from lightning.fabric.loggers import TensorBoardLogger
from lightning.fabric.fabric import _FabricOptimizer

import torchvision
from box import Box
from datasets import call_load_dataset
from utils.model import Model
from utils.losses import DiceLoss, FocalLoss, Matching_Loss
from utils.eval_utils import AverageMeter, validate, get_prompts, calc_iou, validate_sam2
from utils.tools import copy_model, create_csv, reduce_instances
from utils.utils import *

import  csv, copy
import torch
import torch.nn.functional as F
from collections import deque


import os
import numpy as np
from tqdm import tqdm
from scipy.ndimage import map_coordinates

import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader

import lightning as L

import segmentation_models_pytorch as smp

from box import Box
from utils.model import Model
from utils.sample_utils import get_point_prompts
from utils.tools import write_csv

/home/tic/miniconda3/envs/cvpr/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
    

# %% [markdown]
# ## Optimizer and Scheduler Configuration

# %% [code]
def configure_opt(cfg: Box, model: Model):
    def lr_lambda(step):
        if step < cfg.opt.warmup_steps:
            return step / cfg.opt.warmup_steps
        elif step < cfg.opt.steps[0]:
            return 1.0
        elif step < cfg.opt.steps[1]:
            return 1 / cfg.opt.decay_factor
        else:
            return 1 / (cfg.opt.decay_factor**2)

    optimizer = torch.optim.Adam(model.model.parameters(),
                                 lr=cfg.opt.learning_rate,
                                 weight_decay=cfg.opt.weight_decay)
    scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)
    return optimizer, scheduler





In [3]:
# %% [markdown]
# ## Configuration Loading and Launch

# %% [code]
# Example: set arguments manually here
# Replace with your config module path, e.g. "configs.default_config"
import importlib

CFG_MODULE = "configs.config_nwpu"
cfg_module = importlib.import_module(CFG_MODULE)
cfg = cfg_module.cfg

# Manually merge updates if needed
cfg.out_dir = "./outputs"
cfg.resume = False

torch.cuda.empty_cache()
torch.set_float32_matmul_precision('high')

# main(cfg)

In [4]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import torchvision.transforms.functional as TF
import cv2
from matplotlib.patches import Rectangle


import torch
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

import torch
import numpy as np
import matplotlib.pyplot as plt

def plot_instance_overlap(seg_tensor: torch.Tensor, image: np.ndarray, alpha=1):
    """
    Overlay colored instance masks only on masked regions of the image.

    Args:
        seg_tensor (torch.Tensor): Segmentation tensor of shape (N, 1, H, W),
                                   where N = number of instances.
        image (np.ndarray): Original image, shape (H, W, 3) or (H, W).
        alpha (float): Transparency for overlay (0 = transparent, 1 = opaque).
    """
    # Convert tensor to numpy and squeeze
    seg_np = seg_tensor.squeeze(1).cpu().numpy()  # shape: (N, H, W)
    num_instances, H, W = seg_np.shape

    # Normalize image if needed
    img = image.astype(np.float32)
    if img.max() > 1:
        img = img / 255.0
    if img.ndim == 2:  # grayscale → RGB
        img = np.stack([img] * 3, axis=-1)

    # Copy image to draw overlays
    overlay = img.copy()

    # Generate random distinct colors for each instance
    rng = np.random.default_rng(42)
    colors = rng.uniform(0, 1, size=(num_instances, 3))

    for i in range(num_instances):
        mask = seg_np[i] > 0.5  # binary mask
        color = colors[i]

        # Apply color only to masked regions (blend only on mask)
        for c in range(3):
            overlay[..., c][mask] = (
                (1 - alpha) * img[..., c][mask] + alpha * color[c]
            )

    return overlay


import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

# def visualize_points_and_boxes(img_np, prompts, H, W, num_instances):
#     """
#     Draw image with:
#       1) Red points (positive only)
#       2) Green bounding boxes (separately)
#     """

#     # Convert bounding boxes to pixel scale if normalized
#     # if bboxes.max() <= 1.5:
#     #     bboxes_scaled = bboxes.clone()
#     #     bboxes_scaled[:, [0, 2]] *= W
#     #     bboxes_scaled[:, [1, 3]] *= H
#     # else:
#     #     bboxes_scaled = bboxes

#     # --- Get prompt coordinates and labels
#     try:
#         point_coords = prompts[0][0].detach().cpu().numpy()  # [N, 2]
#         point_labels = prompts[0][1].detach().cpu().numpy()  # [N,]
#         # scale normalized coordinates if needed
#         if point_coords.max() <= 1.5:
#             point_coords_px = point_coords.copy()
#             point_coords_px[:, 0] *= W
#             point_coords_px[:, 1] *= H
#         else:
#             point_coords_px = point_coords
#     except Exception as e:
#         print(f"Could not extract prompts: {e}")
#         point_coords_px, point_labels = np.empty((0, 2)), np.empty((0,))

#     # --- (1) IMAGE + POSITIVE POINTS (RED)
#     fig1, ax1 = plt.subplots(figsize=(10, 10))
#     ax1.imshow(img_np)
#     pos_points = point_coords_px[point_labels == 1]
#     if len(pos_points) > 0:
#         ax1.scatter(
#             pos_points[:, 0], pos_points[:, 1],
#             c='red', s=100, marker='o', edgecolors='white',
#             linewidths=1.5, label='Positive Points', zorder=5
#         )
#     ax1.set_title("Image with Positive Points (Red)")
#     ax1.axis("off")
#     if len(pos_points) > 0:
#         ax1.legend(loc='upper right')
#     plt.tight_layout()
#     plt.show()

#     # --- (2) IMAGE + BOUNDING BOXES (GREEN)
#     fig2, ax2 = plt.subplots(figsize=(10, 10))
#     ax2.imshow(img_np)
#     # for idx, box in enumerate(bboxes_scaled):
#     #     x1, y1, x2, y2 = box.tolist()
#     #     width, height = x2 - x1, y2 - y1
#     #     rect = Rectangle(
#     #         (x1, y1), width, height,
#     #         linewidth=2, edgecolor='lime', facecolor='none',
#     #         label='Bounding Box' if idx == 0 else ""
#     #     )
#     #     ax2.add_patch(rect)
#     ax2.set_title(f"Image with Bounding Boxes (Green) — {num_instances} instances")
#     ax2.axis("off")
#     handles, labels = ax2.get_legend_handles_labels()
#     if handles:
#         ax2.legend(handles, labels, loc='upper right')
#     plt.tight_layout()
#     plt.show()


import numpy as np
import matplotlib
matplotlib.use("Agg")  # Safe backend for headless / script mode
import matplotlib.pyplot as plt

def visualize_points(img_np, prompts, H, W, show=True):
    """
    Draw image with:
      🔴 Red positive points only.
    Returns:
      img_points_vis (np.ndarray): RGB image with drawn points.
    """

    # --- Extract prompt coordinates safely ---
    try:
        point_coords = prompts[0][0].detach().cpu().numpy()  # [N, 2]
        point_labels = prompts[0][1].detach().cpu().numpy()  # [N,]
        # scale normalized coordinates if needed
        if point_coords.max() <= 1.5:
            point_coords_px = point_coords.copy()
            point_coords_px[:, 0] *= W
            point_coords_px[:, 1] *= H
        else:
            point_coords_px = point_coords
    except Exception as e:
        print(f"Could not extract prompts: {e}")
        point_coords_px, point_labels = np.empty((0, 2)), np.empty((0,))

    # --- Draw positive points ---
    fig, ax = plt.subplots(figsize=(10, 10))
    ax.imshow(img_np)
    pos_points = point_coords_px[point_labels == 1]

    if len(pos_points) > 0:
        ax.scatter(
            pos_points[:, 0], pos_points[:, 1],
            c='red', s=100, marker='o', edgecolors='white',
            linewidths=1.5, zorder=5
        )
        ax.legend(loc='upper right')

    ax.set_title("Image with Positive Points (Red)")
    ax.axis("off")
    fig.tight_layout()

    # --- Convert figure to a NumPy image safely ---
    fig.canvas.draw()
    img_rgba = np.asarray(fig.canvas.buffer_rgba())
    img_points_vis = img_rgba[:, :, :3].copy()  # drop alpha channel

    if show:
        plt.show()
    else:
        plt.close(fig)

    return img_points_vis


import cv2
import torch
import numpy as np
import matplotlib.pyplot as plt

def visualize_per_instance(img_np, gt_masks_np, entropy_maps_np, preds_np, pseudo_lab):
    """
    Visualize per-instance segmentation details:
      - Ground truth mask (overlay)
      - Entropy map
      - Prediction mask (overlay)
    Then show global prediction + pseudo labels.
    """
    num_instances = gt_masks_np.shape[0]
    H, W = gt_masks_np.shape[-2:]
    print(f"🔹 Visualizing {num_instances} instances, each {H}×{W}")

    for i in range(num_instances):
        gt_mask = gt_masks_np[i]
        entropy = entropy_maps_np[i][0]
        pred_mask = preds_np[i][0]

        # Normalize entropy to [0,1]
        entropy_norm = (entropy - entropy.min()) / (entropy.max() - entropy.min() + 1e-8)

        # === GT mask overlay ===
        gt_mask_rgb = cv2.applyColorMap((gt_mask * 255).astype(np.uint8), cv2.COLORMAP_JET)
        gt_mask_rgb = cv2.cvtColor(gt_mask_rgb, cv2.COLOR_BGR2RGB)
        gt_overlay = cv2.addWeighted(gt_mask_rgb, 0.5, img_np, 0.5, 0)

        # === Entropy heatmap ===
        entropy_heatmap = cv2.applyColorMap((entropy_norm * 255).astype(np.uint8), cv2.COLORMAP_JET)
        entropy_heatmap = cv2.cvtColor(entropy_heatmap, cv2.COLOR_BGR2RGB)

        # === Pred mask overlay ===
        pred_mask_rgb = cv2.applyColorMap((pred_mask * 255).astype(np.uint8), cv2.COLORMAP_PARULA)
        pred_mask_rgb = cv2.cvtColor(pred_mask_rgb, cv2.COLOR_BGR2RGB)
        pred_overlay = cv2.addWeighted(pred_mask_rgb, 0.5, img_np, 0.5, 0)

        # --- Plot instance summary ---
        fig, axes = plt.subplots(1, 3, figsize=(12, 12))
        fig.suptitle(f"Instance {i+1}", fontsize=18)

        axes[0].imshow(gt_overlay)
        axes[0].set_title("GT Mask Overlay", fontsize=14)
        axes[0].axis("off")

        axes[1].imshow(entropy_heatmap)
        axes[1].set_title("Entropy Map", fontsize=14)
        axes[1].axis("off")

        axes[2].imshow(pred_overlay)
        axes[2].set_title("Prediction Overlay", fontsize=14)
        axes[2].axis("off")

        plt.tight_layout()
        plt.show()

    # --- Global visualization ---
    print("✅ Global Prediction & Pseudo Labels")

    # Combine predictions across instances
    pred_all = (preds_np > 0.5).sum(axis=0)[0]
    pseudo_np = (pseudo_lab[0].detach().cpu().numpy() > 0.5)

    fig, axes = plt.subplots(1, 2, figsize=(12, 12))
    axes[0].imshow(pred_all, cmap='gray')
    axes[0].set_title("Combined Predictions (>0.5)", fontsize=14)
    axes[0].axis("off")

    axes[1].imshow(pseudo_np, cmap='gray')
    axes[1].set_title("Pseudo Labels (>0.5)", fontsize=14)
    axes[1].axis("off")

    plt.tight_layout()
    plt.show()


idx = 1
idx = 1
def visualize_instance_segmentation(images, gt_masks_new,prompts, preds,idx):


    # --- Convert image tensor to numpy
    img = images[0].detach().cpu()
    mean = torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1)
    std = torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1)
    if img.min() < 0:
        img = img * std + mean
    img_np = TF.to_pil_image(img.clamp(0, 1))
    img_np = img_np.resize((1024, 1024), Image.BILINEAR)
    
    img_np = np.array(img_np)
    
    
    # Function to convert matplotlib fig to np image
    def fig_to_np(fig):
        fig.canvas.draw()
        img = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
        img = img.reshape(fig.canvas.get_width_height()[::-1] + (3,))
        plt.close(fig)
        return img
    
    # Example index
      # change this dynamically in your loop
    
    # === Visualize and save ===
    
    # 1️⃣ Visualize points (input image with prompts)
    H, W = gt_masks_new[0].shape[-2:]
    img_vis = visualize_points(img_np, prompts, H, W, show=False)
    fig1, ax1 = plt.subplots()
    ax1.imshow(img_vis)
    ax1.axis('off')
    fig1.savefig(os.path.join(save_dir, f"{idx:03d}_a_img_vis.png"), bbox_inches='tight', pad_inches=0)
    plt.close(fig1)
    
    # 2️⃣ Overlay Ground Truth
    overlay_gt = plot_instance_overlap(gt_masks_new[0], img_np)
    fig2, ax2 = plt.subplots()
    ax2.imshow(overlay_gt)
    ax2.axis('off')
    fig2.savefig(os.path.join(save_dir, f"{idx:03d}_b_gt.png"), bbox_inches='tight', pad_inches=0)
    plt.close(fig2)
    
    # 3️⃣ Overlay Prediction
    overlay_pred = plot_instance_overlap(preds.squeeze(1), img_np)
    fig3, ax3 = plt.subplots()
    ax3.imshow(overlay_pred)
    ax3.axis('off')
    fig3.savefig(os.path.join(save_dir, f"{idx:03d}_c_{method}.png"), bbox_inches='tight', pad_inches=0)
    plt.close(fig3)
    idx+=1
        


    # pseudo_lab = torch.sigmoid(torch.stack(soft_masks, dim=0)).sum(dim=1)
    # visualize_per_instance(img_np, gt_masks_np, entropy_maps_np, preds_np, pseudo_lab)



In [5]:
idx = 1
import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import torchvision.transforms.functional as TF
import os

idx = 1

def visualsize_overlapa(images, gt_masks_new, prompts, preds,idx, save_dir, method="pred_overlap"):

    # --- Convert image tensor to numpy
    img = images[0].detach().cpu()
    mean = torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1)
    std = torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1)
    if img.min() < 0:
        img = img * std + mean
    img_np = TF.to_pil_image(img.clamp(0, 1))
    img_np = img_np.resize((1024, 900), Image.BILINEAR)
    img_np = np.array(img_np)

    

    # === Compute overlap across instances ===
    # preds assumed shape: [num_instances, H, W]
    instance_masks = preds.squeeze(1).cpu().numpy()  # [N, H, W]
    if instance_masks.ndim == 2:
        instance_masks = np.expand_dims(instance_masks, axis=0)  # handle single instance case
    mean_thresh = 0.5
   
    pred_binary = (((instance_masks)>mean_thresh) )
    pred_binary = torch.tensor(pred_binary)  # convert numpy array to tensor
    overlap_count = pred_binary.sum(dim=0)   # now dim works
    overlap_mask = (overlap_count > 1)
    
    overlap_count = pred_binary.sum(dim=0)
    overlap_mask = (overlap_count > 1)



    total_foreground = (pred_binary > 0).sum()
    overlap_pixels = overlap_mask.sum()
    overlap_ratio = overlap_pixels / (total_foreground + 1e-8)
    print(overlap_ratio)
    if overlap_ratio > 0.01:

        # === Visualize input prompts ===
        H, W = gt_masks_new[0].shape[-2:]
        img_vis = visualize_points(img_np, prompts, H, W, show=False)
        fig1, ax1 = plt.subplots()
        ax1.imshow(img_vis)
        ax1.axis('off')
        fig1.savefig(os.path.join(save_dir, f"{idx:03d}_a_img_vis.png"), bbox_inches='tight', pad_inches=0)
        plt.close(fig1)
        
        # === Overlay prediction ===
        overlay_pred = plot_instance_overlap(preds.squeeze(1), img_np)
        fig3, ax3 = plt.subplots()
        ax3.imshow(overlay_pred)
        ax3.axis('off')
        fig3.savefig(os.path.join(save_dir, f"{idx:03d}_c_{method}.png"), bbox_inches='tight', pad_inches=0)
        plt.close(fig3)

        
        # # Keep only pixels with >50% overlap
        # overlap_mask = (overlap_map / num_instances) > overlap_threshold
    
        # --- Overlay overlap mask on input image ---
        overlay_img = img_np.copy()
        overlay_color = np.array([255, 0, 0], dtype=np.uint8)  # Red for overlap
        overlay_img[overlap_mask] = (0.5 * overlay_img[overlap_mask] + 0.5 * overlay_color).astype(np.uint8)
    
        # Save overlap visualization
        fig4, ax4 = plt.subplots()
        ax4.imshow(overlay_img)
        ax4.axis('off')
        fig4.savefig(os.path.join(save_dir, f"{idx:03d}_overlap.png"), bbox_inches='tight', pad_inches=0)
        plt.close(fig4)
    
        idx += 1


In [6]:
idx=1
def validate_w_show(fabric: L.Fabric, cfg: Box, model: Model, val_dataloader: DataLoader, name: str, epoch: int = 0):
    model.eval()
    ious = AverageMeter()
    f1_scores = AverageMeter()
    recall = AverageMeter()
    precision = AverageMeter()
    idx = 1
    with torch.no_grad():
        for iter, data in enumerate(tqdm(val_dataloader, desc='Validation', ncols=100)):
            images, bboxes, gt_masks, img_paths = data
            num_images = images.size(0)
            prompts = get_prompts(cfg, bboxes, gt_masks)

            _, pred_masks, _, _ = model(images, prompts)

            
            # visualsize_overlapa(images, gt_masks,prompts,  pred_masks[0], idx,save_dir)
            
            
            visualize_instance_segmentation(images, gt_masks,prompts,  pred_masks[0], idx)
            for pred_mask, gt_mask in zip(pred_masks, gt_masks):
                batch_stats = smp.metrics.get_stats(
                    pred_mask,
                    gt_mask.int(),
                    mode='binary',
                    threshold=0.5,
                )
                batch_recall = smp.metrics.recall(*batch_stats, reduction="micro-imagewise")
                batch_precision = smp.metrics.precision(*batch_stats, reduction="micro-imagewise")
                batch_iou = smp.metrics.iou_score(*batch_stats, reduction="micro-imagewise")
                batch_f1 = smp.metrics.f1_score(*batch_stats, reduction="micro-imagewise")
                ious.update(batch_iou, num_images)
                f1_scores.update(batch_f1, num_images)
                recall.update(batch_recall, num_images)
                precision.update(batch_precision, num_images)

            torch.cuda.empty_cache()
            idx+=1
            if idx > 7:
                break

    fabric.print(
        f'Val: [{epoch}] - [{iter+1}/{len(val_dataloader)}]: IoU: [{ious.avg:.4f}] -- Recall: [{recall.avg:.4f}] -- Precision [{precision.avg:.4f}] -- F1: [{f1_scores.avg:.4f}]'
    )
    csv_dict = {"Prompt": cfg.prompt, "IoU": f"{ious.avg:.4f}","Recall": f"{recall.avg:.4f}", "Precision": f"{precision.avg:.4f}", "F1": f"{f1_scores.avg:.4f}", "epoch": epoch}

    if fabric.global_rank == 0:
        write_csv(os.path.join(cfg.out_dir, "metrics.csv"), csv_dict, csv_head=cfg.csv_keys)
    return ious.avg, f1_scores.avg

In [7]:
gpu_ids = [str(i) for i in range(torch.cuda.device_count())]
num_devices = len(gpu_ids)
fabric = L.Fabric(accelerator="auto",
                  devices=num_devices,
                  strategy="auto",
                  loggers=[TensorBoardLogger(cfg.out_dir)])
fabric.launch()
fabric.seed_everything(1337 + fabric.global_rank)

if fabric.global_rank == 0:
    os.makedirs(os.path.join(cfg.out_dir, "save"), exist_ok=True)
    create_csv(os.path.join(cfg.out_dir, "metrics.csv"), csv_head=cfg.csv_keys)

with fabric.device:
    model = Model(cfg)
    model.setup()

load_datasets = call_load_dataset(cfg)
train_data, val_data, pt_data = load_datasets(cfg, img_size=1024, return_pt = True)
train_data = fabric._setup_dataloader(train_data)
val_data = fabric._setup_dataloader(val_data)
pt_data = fabric._setup_dataloader(pt_data)
optimizer, scheduler = configure_opt(cfg, model)
model, optimizer = fabric.setup(model, optimizer)



auto_ckpt = './pretrained_all/nwpu/resam/best_model.pth'#
method = "sam"
data_= "nwpu"
save_dir = "results"+f"/{data_}/"
os.makedirs(save_dir, exist_ok=True)
print(auto_ckpt)
if auto_ckpt is not None:
    full_checkpoint = fabric.load(auto_ckpt)

    if isinstance(full_checkpoint, dict) and "model" in full_checkpoint:
        model.load_state_dict(full_checkpoint["model"])
        if "optimizer" in full_checkpoint:
            optimizer.load_state_dict(full_checkpoint["optimizer"])
    else:
        model.load_state_dict(full_checkpoint)
    loaded = True
    fabric.print(f"Resumed from explicit checkpoint: {cfg.model.ckpt}")

 

Global seed set to 1337


loading annotations into memory...
Done (t=0.02s)
creating index...
index created!
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
loading annotations into memory...
Done (t=0.43s)
creating index...
index created!
./pretrained_all/nwpu/resam/best_model.pth
Resumed from explicit checkpoint: 


In [8]:
init_iou = 0
idx = 1
print('-'*100)
    
print('\033[92mDirect test on the original SAM.\033[0m') 
init_iou, _, = validate_w_show(fabric, cfg, model, val_data, name=cfg.name, epoch=0)
print('-'*100)
del _    

----------------------------------------------------------------------------------------------------
Direct test on the original SAM.


Validation:   0%|                                                           | 0/130 [00:00<?, ?it/s]/tmp/ipykernel_295927/3157961382.py:169: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  ax.legend(loc='upper right')
Validation:   5%|██▎                                                | 6/130 [01:22<28:22, 13.73s/it]

Val: [0] - [7/130]: IoU: [0.3103] -- Recall: [0.9365] -- Precision [0.3205] -- F1: [0.4393]
----------------------------------------------------------------------------------------------------


In [8]:
def process_forward(img_tensor, prompt, model):
    with torch.no_grad():
        _, masks_pred, _, _ = model(img_tensor, prompt)
    entropy_maps = []
    pred_ins = []
    eps=1e-8
    for i, mask_p in enumerate( masks_pred[0]):
        mask_p = torch.sigmoid(mask_p)
        p = mask_p.clamp(1e-6, 1 - 1e-6)
        if p.ndim == 2:
            p = p.unsqueeze(0)

        # entropy_map = entropy_map_calculate(p)
        entropy = - (p * torch.log(p + eps) + (1 - p) * torch.log(1 - p + eps))
        max_ent = torch.log(torch.tensor(2.0, device=mask_p.device))
        entropy_norm = entropy / (max_ent + 1e-8)   # [0, 1]
        entropy_maps.append(entropy_norm)
        pred_ins.append(p)

    return entropy_maps, pred_ins

In [9]:
def entropy_map_calculate(p):
    eps = 1e-8
    p = p.clamp(eps, 1 - eps)  # Safe!
    entropy_map = - (p * torch.log(p) + (1 - p) * torch.log(1 - p))
    # entropy_map = entropy_map.max(dim=0)[0]
    return entropy_map# / torch.log(torch.tensor(2.0))

In [10]:
def get_bbox_feature(embedding_map, bbox, stride=16, pooling='avg'):
    """
    Extract a feature vector from an embedding map given a bounding box.
    
    Args:
        embedding_map (torch.Tensor): Shape (C, H_feat, W_feat) or (B, C, H_feat, W_feat)
        bbox (list or torch.Tensor): [x1, y1, x2, y2] in original image coordinates
        stride (int): Downscaling factor between image and feature map
        pooling (str): 'avg' or 'max' pooling inside the bbox region
        
    Returns:
        torch.Tensor: Feature vector of shape (C,)
    """
    # If batch dimension exists, assume batch size 1
    if embedding_map.dim() == 4:
        embedding_map = embedding_map[0]

    C, H_feat, W_feat = embedding_map.shape
    x1, y1, x2, y2 = bbox

    # Map bbox to feature map coordinates
    fx1 = max(int(x1 / stride), 0)
    fy1 = max(int(y1 / stride), 0)
    fx2 = min(int((x2 + stride - 1) / stride), W_feat)  # ceil division
    fy2 = min(int((y2 + stride - 1) / stride), H_feat)

    # Crop the feature map to bbox region
    region = embedding_map[:, fy1:fy2, fx1:fx2]

    if region.numel() == 0:
        # fallback to global feature if bbox is too small
        region = embedding_map

    # Pool to get a single feature vector
    if pooling == 'avg':
        feature_vec = region.mean(dim=(1,2))
    elif pooling == 'max':
        feature_vec = region.amax(dim=(1,2))
    else:
        raise ValueError("pooling must be 'avg' or 'max'")

    return feature_vec

In [11]:
def configure_opt(cfg: Box, model: Model):

    def lr_lambda(step):
        if step < cfg.opt.warmup_steps:
            return step / cfg.opt.warmup_steps
        elif step < cfg.opt.steps[0]:
            return 1.0
        elif step < cfg.opt.steps[1]:
            return 1 / cfg.opt.decay_factor
        else:
            return 1 / (cfg.opt.decay_factor**2)

    # optimize only trainable params (e.g., LoRA)
    trainable_params = (p for p in model.model.parameters() if p.requires_grad)
    optimizer = torch.optim.Adam(trainable_params, lr=cfg.opt.learning_rate, weight_decay=cfg.opt.weight_decay)
    scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)

    return optimizer, scheduler

In [12]:
def similarity_loss(features, queue, tau=0.07):
    """
    features: [B, D] current batch embeddings (normalized)
    queue: deque of [D] past embeddings (detached)
    """
    if len(queue) == 0:
        return torch.tensor(0., device=features.device)

    # Stack all past features from queue
    with torch.no_grad():
        past_feats = torch.stack(list(queue), dim=0)  # [Q, D]

    # Normalize
    features = F.normalize(features, dim=1)
    past_feats = F.normalize(past_feats, dim=1)

    # Compute cosine similarities (batch x queue)
    logits = torch.mm(features, past_feats.t()) / tau  # [B, Q]
    probs = F.softmax(logits, dim=1)

    # Weighted alignment (like SSAL)
    cos = (logits * tau).clamp(-1, 1)  # revert scaling, approximate cos
    loss = ((1 - cos) * probs).sum(dim=1).mean()

    return loss

In [13]:
def _find_latest_checkpoint(save_dir):
    """
    Look for the most recent .pt/.pth file in save_dir.
    Returns absolute path or None if not found.
    """
    if not os.path.isdir(save_dir):
        return None
    ckpt_files = [
        os.path.join(save_dir, f)
        for f in os.listdir(save_dir)
        if f.endswith(".pt") or f.endswith(".pth")
    ]
    if not ckpt_files:
        return None
    ckpt_files.sort(key=lambda p: os.path.getmtime(p), reverse=True)
    return ckpt_files[0]


In [14]:
import matplotlib.pyplot as plt
import torchvision.transforms.functional as TF
import torch

In [15]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import torchvision.transforms.functional as TF
import cv2
from matplotlib.patches import Rectangle


import torch
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

import torch
import numpy as np
import matplotlib.pyplot as plt

def plot_instance_overlap(seg_tensor: torch.Tensor, image: np.ndarray, alpha=1):
    """
    Overlay colored instance masks only on masked regions of the image.

    Args:
        seg_tensor (torch.Tensor): Segmentation tensor of shape (N, 1, H, W),
                                   where N = number of instances.
        image (np.ndarray): Original image, shape (H, W, 3) or (H, W).
        alpha (float): Transparency for overlay (0 = transparent, 1 = opaque).
    """
    # Convert tensor to numpy and squeeze
    seg_np = seg_tensor.squeeze(1).cpu().numpy()  # shape: (N, H, W)
    num_instances, H, W = seg_np.shape

    # Normalize image if needed
    img = image.astype(np.float32)
    if img.max() > 1:
        img = img / 255.0
    if img.ndim == 2:  # grayscale → RGB
        img = np.stack([img] * 3, axis=-1)

    # Copy image to draw overlays
    overlay = img.copy()

    # Generate random distinct colors for each instance
    rng = np.random.default_rng(42)
    colors = rng.uniform(0, 1, size=(num_instances, 3))

    for i in range(num_instances):
        mask = seg_np[i] > 0.5  # binary mask
        color = colors[i]

        # Apply color only to masked regions (blend only on mask)
        for c in range(3):
            overlay[..., c][mask] = (
                (1 - alpha) * img[..., c][mask] + alpha * color[c]
            )

    plt.figure(figsize=(12, 9))
    plt.imshow(overlay)
    plt.axis("off")
    plt.title(f"Instance Overlay ({num_instances} instances)")
    plt.show()


import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

def visualize_points_and_boxes(img_np, bboxes, prompts, H, W, num_instances):
    """
    Draw image with:
      1) Red points (positive only)
      2) Green bounding boxes (separately)
    """

    # Convert bounding boxes to pixel scale if normalized
    if bboxes.max() <= 1.5:
        bboxes_scaled = bboxes.clone()
        bboxes_scaled[:, [0, 2]] *= W
        bboxes_scaled[:, [1, 3]] *= H
    else:
        bboxes_scaled = bboxes

    # --- Get prompt coordinates and labels
    try:
        point_coords = prompts[0][0].detach().cpu().numpy()  # [N, 2]
        point_labels = prompts[0][1].detach().cpu().numpy()  # [N,]
        # scale normalized coordinates if needed
        if point_coords.max() <= 1.5:
            point_coords_px = point_coords.copy()
            point_coords_px[:, 0] *= W
            point_coords_px[:, 1] *= H
        else:
            point_coords_px = point_coords
    except Exception as e:
        print(f"Could not extract prompts: {e}")
        point_coords_px, point_labels = np.empty((0, 2)), np.empty((0,))

    # --- (1) IMAGE + POSITIVE POINTS (RED)
    fig1, ax1 = plt.subplots(figsize=(10, 7))
    ax1.imshow(img_np)
    pos_points = point_coords_px[point_labels == 1]
    if len(pos_points) > 0:
        ax1.scatter(
            pos_points[:, 0], pos_points[:, 1],
            c='red', s=100, marker='o', edgecolors='white',
            linewidths=1.5, label='Positive Points', zorder=5
        )
    ax1.set_title("Image with Positive Points (Red)")
    ax1.axis("off")
    if len(pos_points) > 0:
        ax1.legend(loc='upper right')
    plt.tight_layout()
    plt.show()

    # --- (2) IMAGE + BOUNDING BOXES (GREEN)
    fig2, ax2 = plt.subplots(figsize=(10, 7))
    ax2.imshow(img_np)
    for idx, box in enumerate(bboxes_scaled):
        x1, y1, x2, y2 = box.tolist()
        width, height = x2 - x1, y2 - y1
        rect = Rectangle(
            (x1, y1), width, height,
            linewidth=2, edgecolor='lime', facecolor='none',
            label='Bounding Box' if idx == 0 else ""
        )
        ax2.add_patch(rect)
    ax2.set_title(f"Image with Bounding Boxes (Green) — {num_instances} instances")
    ax2.axis("off")
    handles, labels = ax2.get_legend_handles_labels()
    if handles:
        ax2.legend(handles, labels, loc='upper right')
    plt.tight_layout()
    plt.show()

import cv2
import torch
import numpy as np
import matplotlib.pyplot as plt

def visualize_per_instance(img_np, gt_masks_np, entropy_maps_np, preds_np, pseudo_lab):
    """
    Visualize per-instance segmentation details:
      - Ground truth mask (overlay)
      - Entropy map
      - Prediction mask (overlay)
    Then show global prediction + pseudo labels.
    """
    num_instances = gt_masks_np.shape[0]
    H, W = gt_masks_np.shape[-2:]
    print(f"🔹 Visualizing {num_instances} instances, each {H}×{W}")

    for i in range(num_instances):
        gt_mask = gt_masks_np[i]
        entropy = entropy_maps_np[i][0]
        pred_mask = preds_np[i][0]

        # Normalize entropy to [0,1]
        entropy_norm = (entropy - entropy.min()) / (entropy.max() - entropy.min() + 1e-8)

        # === GT mask overlay ===
        gt_mask_rgb = cv2.applyColorMap((gt_mask * 255).astype(np.uint8), cv2.COLORMAP_JET)
        gt_mask_rgb = cv2.cvtColor(gt_mask_rgb, cv2.COLOR_BGR2RGB)
        gt_overlay = cv2.addWeighted(gt_mask_rgb, 0.5, img_np, 0.5, 0)

        # === Entropy heatmap ===
        entropy_heatmap = cv2.applyColorMap((entropy_norm * 255).astype(np.uint8), cv2.COLORMAP_JET)
        entropy_heatmap = cv2.cvtColor(entropy_heatmap, cv2.COLOR_BGR2RGB)

        # === Pred mask overlay ===
        pred_mask_rgb = cv2.applyColorMap((pred_mask * 255).astype(np.uint8), cv2.COLORMAP_PARULA)
        pred_mask_rgb = cv2.cvtColor(pred_mask_rgb, cv2.COLOR_BGR2RGB)
        pred_overlay = cv2.addWeighted(pred_mask_rgb, 0.5, img_np, 0.5, 0)

        # --- Plot instance summary ---
        fig, axes = plt.subplots(1, 3, figsize=(12, 12))
        fig.suptitle(f"Instance {i+1}", fontsize=18)

        axes[0].imshow(gt_overlay)
        axes[0].set_title("GT Mask Overlay", fontsize=14)
        axes[0].axis("off")

        axes[1].imshow(entropy_heatmap)
        axes[1].set_title("Entropy Map", fontsize=14)
        axes[1].axis("off")

        axes[2].imshow(pred_overlay)
        axes[2].set_title("Prediction Overlay", fontsize=14)
        axes[2].axis("off")

        plt.tight_layout()
        plt.show()

    # --- Global visualization ---
    print("✅ Global Prediction & Pseudo Labels")

    # Combine predictions across instances
    pred_all = (preds_np > 0.5).sum(axis=0)[0]
    pseudo_np = (pseudo_lab[0].detach().cpu().numpy() > 0.5)

    fig, axes = plt.subplots(1, 2, figsize=(12, 12))
    axes[0].imshow(pred_all, cmap='gray')
    axes[0].set_title("Combined Predictions (>0.5)", fontsize=14)
    axes[0].axis("off")

    axes[1].imshow(pseudo_np, cmap='gray')
    axes[1].set_title("Pseudo Labels (>0.5)", fontsize=14)
    axes[1].axis("off")

    plt.tight_layout()
    plt.show()



def visualize_instance_segmentation(images_weak, gt_masks_new,prompts, entropy_maps, preds,
                                    invert_overlap_map, soft_masks, bboxes, j=0, slice_step=50):
    """
    Visualizes:
    1. The weak input image
    2. Ground-truth instance masks
    3. Entropy heatmaps
    4. Predicted masks
    5. Bounding boxes overlaid on image


    
    """


    # --- Convert image tensor to numpy
    img = images_weak[0].detach().cpu()
    mean = torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1)
    std = torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1)
    if img.min() < 0:
        img = img * std + mean
    img_np = TF.to_pil_image(img.clamp(0, 1))
    img_np = img_np.resize((1024, 1024), Image.BILINEAR)
    
    img_np = np.array(img_np)
    print("DDDDDDDDDDDDDDD", img_np.shape)
    print(f"Orignal Image")


    
    plt.figure(figsize=(12, 12))
    plt.imshow(img_np)
    plt.show()

    
    

    print("Ground Truth")
    plot_instance_overlap(gt_masks_new[0], img_np)


    print("Current Model Output")
    plot_instance_overlap(preds.squeeze(1), img_np)


    num_instances = gt_masks_new.shape[1]
    H, W = gt_masks_new.shape[-2:]
    
    visualize_points_and_boxes(img_np, bboxes, prompts, H, W, num_instances)


    # pseudo_lab = torch.sigmoid(torch.stack(soft_masks, dim=0)).sum(dim=1)
    # visualize_per_instance(img_np, gt_masks_np, entropy_maps_np, preds_np, pseudo_lab)

    preds_np = preds.detach().cpu().numpy()
    # num_instances = soft_masks[0].shape[0]

    soft_masks_np = soft_masks[0]

  
    

    # --- Visualize per-instance details
    n_cols = 4
    n_rows = min(num_instances, 5)
    for i in range(n_rows):
        pred_mask = preds_np[i][0]
      


        print(f"Instance {i+1} Predicted mask")
        plt.imshow(pred_mask > 0.5, cmap='gray')
        plt.show()


        soft_mask = soft_masks_np[i]
        soft_mask = torch.sigmoid(soft_mask)


        print(f"Pseudo {i+1} mask")
        plt.imshow(soft_mask.detach().cpu().numpy() > 0.5, cmap='gray')
        plt.show()

        entropy_map = entropy_maps[i]
        print(f"Entropy {i+1} mask")
        plt.imshow(entropy_map[0].detach().cpu().numpy() , cmap='viridis')
        plt.show()

    

    
    
    print(f"INVERTED overlap  mask")
    plt.imshow((1-invert_overlap_map[0]).detach().cpu().numpy() > 0.5, cmap='gray')
    plt.show()

    pred_one = (preds_np > 0.5).sum(axis=0)
    print("Pred all")
    plt.imshow(pred_one[0] > 0.5, cmap='gray')
    plt.show()
    
    print("Pseudo labels")
    pseudo_lab = (soft_masks_np.detach().cpu().numpy() > 0.99).sum(axis=0)
    print(pseudo_lab.shape)
    print(pseudo_lab.max())
    plt.imshow((pseudo_lab > 0.99), cmap='gray')
    plt.show()

    print("Bounding boxes tensor:", bboxes.shape)


In [16]:
import random
feature_queue = deque(maxlen=32) 


In [17]:
def train_sam(
    cfg: Box,
    fabric: L.Fabric,
    model: Model,
    optimizer: _FabricOptimizer,
    scheduler: _FabricOptimizer,
    train_dataloader: DataLoader,
    val_dataloader: DataLoader,
    init_iou, 
    vis
):

    
    # collected = sort_entropy_(model, target_pts)
    focal_loss = FocalLoss()
    dice_loss = DiceLoss()
    best_ent = 1000000
    best_state = copy.deepcopy(model.state_dict())
    no_improve_count = 0
    max_patience = cfg.get("patience", 3)  # stop if no improvement for X validations
    match_interval = cfg.match_interval
    eval_interval = int(len(train_dataloader) * 1)

    window_size = 30

    embedding_queue = []
    ite_em = 0

    # Prepare output dirs
    os.makedirs(os.path.join(cfg.out_dir, "save"), exist_ok=True)
    csv_path = os.path.join(cfg.out_dir, "training_log.csv")

    # Initialize CSV
    with open(csv_path, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["Epoch", "Iteration", "Val_ent", "Best_ent", "Status"])

    fabric.print(f"Training with rollback enabled. Logging to: {csv_path}")

    entropy_means = deque(maxlen=len(train_dataloader))

    eps = 1e-8
    for epoch in range(1, cfg.num_epochs + 1):
        batch_time = AverageMeter()
        data_time = AverageMeter()
        focal_losses = AverageMeter()
        dice_losses = AverageMeter()
        iou_losses = AverageMeter()
        total_losses = AverageMeter()
        match_losses = AverageMeter()
        sim_losses = AverageMeter()
        end = time.time()
        num_iter = len(train_dataloader)

        idx = random.randint(0, len(train_dataloader) - 1)
        data = train_dataloader.dataset[10]
        print(idx)

        for iter, data in enumerate(train_dataloader):

            data_time.update(time.time() - end)
            images_weak, images_strong, bboxes, gt_masks, img_paths= data
            del data

            if iter==yu:
                step_size = 50
                for j in range(0, len(gt_masks[0]), step_size):
                    
                    
                    gt_masks_new = gt_masks[0][j:j+step_size].unsqueeze(0)
                    prompts = get_prompts(cfg, bboxes, gt_masks_new)
    
                    batch_size = images_weak.size(0)
    
                    entropy_maps, preds = process_forward(images_weak, prompts, model)
                    entropy_maps = torch.stack(entropy_maps, dim=0)
                    pred_stack = torch.stack(preds, dim=0)
                    pred_binary = (((pred_stack)>0.7) ).float()  
                    overlap_count = pred_binary.sum(dim=0)
                    overlap_map = (overlap_count > 1).float()
                    invert_overlap_map = 1.0 - overlap_map
    
    
    
                    
    
                    bboxes = []
                    point_list = []
                    point_labels_list = []
                    for i,  pred in enumerate( preds):
                     
                        
                        point_coords = prompts[0][0][i][:].unsqueeze(0)
                        point_coords_lab = prompts[0][1][i][:].unsqueeze(0)
    
                        pred = (pred[0]>0.7)
                        pred_w_overlap = pred * invert_overlap_map[0]
    
                        ys, xs = torch.where(pred_w_overlap > 0.5)
                        if len(xs) > 0 and len(ys) > 0:
                            x_min, x_max = xs.min().item(), xs.max().item()
                            y_min, y_max = ys.min().item(), ys.max().item()
    
                            bboxes.append(torch.tensor([x_min, y_min , x_max, y_max], dtype=torch.float32))
    
                            point_list.append(point_coords)
                            point_labels_list.append(point_coords_lab)
                        
                    if len(bboxes) == 0:
                        continue  # skip if no valid region
    
                    point_ = torch.cat(point_list).squeeze(1)
                    point_labels_ = torch.cat(point_labels_list)
                    new_prompts = [(point_, point_labels_)]
    
                    bboxes = torch.stack(bboxes)
    
                    with torch.no_grad():
                        embeddings, soft_masks, _, _ = model(images_weak, bboxes.unsqueeze(0))
                   
                    
    
                    # sof_mask_prob = torch.sigmoid(torch.stack(soft_masks, dim=0))
                    # entropy_sm = - (sof_mask_prob * torch.log(sof_mask_prob + eps) + (1 - sof_mask_prob) * torch.log(1 - sof_mask_prob + eps))
    
                    # entropy_means.append(entropy_sm.detach().mean().cpu().item())
                    
                    _, pred_masks, iou_predictions, _= model(images_strong, prompts)
                    del _
    
                    num_masks = sum(len(pred_mask) for pred_mask in pred_masks)
                    loss_focal = torch.tensor(0., device=fabric.device)
                    loss_dice = torch.tensor(0., device=fabric.device)
                    loss_iou = torch.tensor(0., device=fabric.device)
                    loss_sim = torch.tensor(0., device=fabric.device)
    
    
                    batch_feats = []  # collect all bbox features in current image
    
                    for bbox in bboxes:
                        feat = get_bbox_feature(embeddings, bbox)
                        batch_feats.append(feat)
    
                    if len(batch_feats) > 0:
                     
                        batch_feats = F.normalize(torch.stack(batch_feats, dim=0), dim=1)
                        loss_sim = similarity_loss(batch_feats, feature_queue)
                  
                        # add new features to queue (detach to avoid backprop)
                        for f in batch_feats:
                            feature_queue.append(f.detach())
                    else:
                        loss_sim = torch.tensor(0., device=embeddings.device)

          

                    visualize_instance_segmentation(images_weak, gt_masks_new, prompts, entropy_maps, pred_stack, invert_overlap_map, soft_masks, bboxes)
    
                   
                 
    
                    for i, (pred_mask, soft_mask, iou_prediction, bbox) in enumerate(
                            zip(pred_masks, soft_masks, iou_predictions, bboxes  )
                        ):  
                            embed_feats = get_bbox_feature( embeddings, bbox)
                            embed_feats = F.normalize(embed_feats, p=2, dim=0)
                            embedding_queue.append(embed_feats)
    
                            if len(embedding_queue) > -1:
                                # Stack all embeddings (num_instances, feature_dim)
                                features = torch.stack(embedding_queue, dim=0)  # [N, D]
                                eps = 1e-8
    
                                # Compute cosine similarity matrix
                                cos_sim_matrix = F.cosine_similarity(
                                    features.unsqueeze(1),  # [N, 1, D]
                                    features.unsqueeze(0),  # [1, N, D]
                                    dim=2,
                                    eps=eps
                                )  # shape [N, N]
    
    
    
                                # Remove self-similarity bias
                                num = features.size(0)
                                mask = (1 - torch.eye(num, device=features.device))
                                cos_sim_matrix = cos_sim_matrix * mask
    
                                # ---- Soft alignment (SSAL) ----
                                # Step 1. Rescale cosine to [0,1]
                                cos_sim_matrix = (cos_sim_matrix + 1) / 2
    
                                # Step 2. Compute temperature-scaled soft distribution
                                tau = 0.05  # you can tune in [0.03–0.1]
                                sim_soft = torch.exp(cos_sim_matrix / tau)
                                prob_matrix = sim_soft / (sim_soft.sum(dim=1, keepdim=True) + eps)
    
                                # Step 3. Soft Semantic Alignment Loss
                                loss_sim = ((1 - cos_sim_matrix) * prob_matrix).sum(dim=1).mean()
    
                            else:
                                loss_sim = torch.tensor(0.0, device=embeddings.device)
                           
                            soft_mask = (soft_mask > 0.).float()
                            # Apply entropy mask to losses
                            loss_focal += focal_loss(pred_mask, soft_mask)  #, entropy_mask=entropy_mask
                            loss_dice += dice_loss(pred_mask, soft_mask)   #, entropy_mask=entropy_mask
                            batch_iou = calc_iou(pred_mask, soft_mask)
                            loss_iou += F.mse_loss(iou_prediction, batch_iou, reduction='sum') / num_masks
    
                            # plt.imshow(pred_mask[0].detach().cpu().numpy(), cmap='viridis')
                            # plt.show()
                            # plt.imshow(soft_mask[0].detach().cpu().numpy(), cmap='viridis')
                            # plt.show()
                    del  pred_masks, iou_predictions 
                #     # loss_dist = loss_dist / num_masks
                #     loss_dice = loss_dice #/ num_masks
                #     loss_focal = loss_focal #/ num_masks
                #     torch.cuda.empty_cache()
    
    
                #     loss_total =  20 * loss_focal +  loss_dice  + loss_iou + 0.1*loss_sim#+ loss_iou  +  +
    
    
    
                #     fabric.backward(loss_total)
    
                #     optimizer.step()
                #     scheduler.step()
                #     optimizer.zero_grad()
                #     torch.cuda.empty_cache()
                #     del  prompts, soft_masks
    
                #     batch_time.update(time.time() - end)
                #     end = time.time()
    
                #     focal_losses.update(loss_focal.item(), batch_size)
            #     dice_losses.update(loss_dice.item(), batch_size)
            #     iou_losses.update(loss_iou.item(), batch_size)
            #     total_losses.update(loss_total.item(), batch_size)
            #     sim_losses.update(loss_sim.item(), batch_size)
            
            #     del loss_dice, loss_iou, loss_focal

            # if (iter+1) % match_interval==0:
            #     fabric.print(
            #         f"Epoch [{epoch}] Iter [{iter + 1}/{len(train_dataloader)}] "
            #         f"| Focal {focal_losses.avg:.4f} | Dice {dice_losses.avg:.4f} | "
            #         f"IoU {iou_losses.avg:.4f} | Sim_loss {sim_losses.avg:.4f} | Total {total_losses.avg:.4f}"
            #     )
            # if (iter+1) % eval_interval == 0:
            #     val_iou, _ = validate(fabric, cfg, model, val_dataloader, cfg.name, epoch)

            #     status = ""
            #     if val_iou > 0:  #best_iou
            #         best_iou = val_iou
            #         best_state = copy.deepcopy(model.state_dict())
            #         torch.save(best_state, os.path.join(cfg.out_dir, "save", "best_model.pth"))
            #         status = "Improved → Model Saved"
            #         no_improve_count = 0
            #     else:
            #         model.load_state_dict(best_state)
            #         no_improve_count += 1
            #         status = f"Rollback ({no_improve_count})"

            #     # Write log entry
            #     with open(csv_path, "a", newline="") as f:
            #         writer = csv.writer(f)
            #         writer.writerow([epoch, iter + 1, val_iou, best_iou, status])

            #     fabric.print(f"Validation IoU={val_iou:.4f} | Best={best_iou:.4f} | {status}")

            #     # Stop if model fails to stabilize
            #     if no_improve_count >= max_patience:
            #         fabric.print(f"Training stopped early after {no_improve_count} failed rollbacks.")
            #         return
            


In [18]:
# %% [markdown]
# ## Configuration Loading and Launch

# %% [code]
# Example: set arguments manually here
# Replace with your config module path, e.g. "configs.default_config"
import importlib

CFG_MODULE = "configs.config_nwpu"
cfg_module = importlib.import_module(CFG_MODULE)
cfg = cfg_module.cfg

# Manually merge updates if needed
cfg.out_dir = "./outputs"
cfg.resume = False

torch.cuda.empty_cache()
torch.set_float32_matmul_precision('high')

# main(cfg)

In [19]:
gpu_ids = [str(i) for i in range(torch.cuda.device_count())]
num_devices = len(gpu_ids)
fabric = L.Fabric(accelerator="auto",
                  devices=num_devices,
                  strategy="auto",
                  loggers=[TensorBoardLogger(cfg.out_dir)])
fabric.launch()
fabric.seed_everything(1337 + fabric.global_rank)

if fabric.global_rank == 0:
    os.makedirs(os.path.join(cfg.out_dir, "save"), exist_ok=True)
    create_csv(os.path.join(cfg.out_dir, "metrics.csv"), csv_head=cfg.csv_keys)

with fabric.device:
    model = Model(cfg)
    model.setup()

load_datasets = call_load_dataset(cfg)
train_data, val_data, pt_data = load_datasets(cfg, img_size=1024, return_pt = True)
train_data = fabric._setup_dataloader(train_data)
val_data = fabric._setup_dataloader(val_data)
pt_data = fabric._setup_dataloader(pt_data)
optimizer, scheduler = configure_opt(cfg, model)
model, optimizer = fabric.setup(model, optimizer)



auto_ckpt = None#"./work_dir/nwpu/resam/point_1/save/best_model.pth"#_find_latest_checkpoint(os.path.join(cfg.out_dir, "save"))

print(auto_ckpt)
if auto_ckpt is not None:
    full_checkpoint = fabric.load(auto_ckpt)

    if isinstance(full_checkpoint, dict) and "model" in full_checkpoint:
        model.load_state_dict(full_checkpoint["model"])
        if "optimizer" in full_checkpoint:
            optimizer.load_state_dict(full_checkpoint["optimizer"])
    else:
        model.load_state_dict(full_checkpoint)
    loaded = True
    fabric.print(f"Resumed from explicit checkpoint: {cfg.model.ckpt}")

init_iou = 0
# print('-'*100)
# print('\033[92mDirect test on the original SAM.\033[0m') 
# init_iou, _, = validate(fabric, cfg, model, val_data, name=cfg.name, epoch=0)
# print('-'*100)
# del _     

Global seed set to 1337


loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
loading annotations into memory...
Done (t=0.16s)
creating index...
index created!
None


In [20]:
yu=654
train_sam(cfg, fabric, model, optimizer, scheduler, train_data, val_data, init_iou, True)



Training with rollback enabled. Logging to: ./outputs/training_log.csv
374
318
490
209
130


KeyboardInterrupt: 

In [ ]:
for iter, data in enumerate(train_data):
    images_weak, images_strong, bboxes, gt_masks, img_paths= data
    print(bboxes)
    if iter>2:
        break
      

In [ ]:
# _, _, = validate(fabric, cfg, model, val_data, name=cfg.name, epoch=0)

In [ ]:
def prompt_calibration(cfg, entrop_map, prompts, point_status):
    point_list = []
    point_labels_list = []
    num_points = cfg.num_points

    for m in range(len(entrop_map)):
        point_coords = prompts[0][0][m][:].unsqueeze(0)
        point_coords_lab = prompts[0][1][m][:].unsqueeze(0)

        # Find high-entropy location
        max_idx = torch.argmax(entrop_map[m])
        y = max_idx // entrop_map[m].shape[1]
        x = max_idx % entrop_map[m].shape[1]
        neg_point_coords = torch.tensor([[x.item(), y.item()]], device=point_coords.device).unsqueeze(0)


        # Combine positive and negative points
        point_coords_all = torch.cat((point_coords, neg_point_coords), dim=1)
        
        # Append a new label (1) to the label tensor
        point_labels_all = torch.cat(
            (point_coords_lab, torch.tensor([[point_status]], device=point_coords.device, dtype=point_coords_lab.dtype)),
            dim=1
        )
        
        point_list.append(point_coords_all)
        point_labels_list.append(point_labels_all)



    point_ = torch.cat(point_list).squeeze(1)
    point_labels_ = torch.cat(point_labels_list)
    new_prompts = [(point_, point_labels_)]
    return new_prompts


In [ ]:
import torch

def entropy_map_calculate(p, eps=1e-8):
    # Clamp to avoid log(0)
    p = torch.clamp(p, eps, 1 - eps)
    
    # Compute binary entropy
    entropy_map = - (p * torch.log(p) + (1 - p) * torch.log(1 - p))
    
    # Normalize to 0–1 (since max entropy = log(2))
    entropy_map = entropy_map / torch.log(torch.tensor(2.0))
    
    return entropy_map


In [ ]:
def process_forward(img_tensor, prompt):
    with torch.no_grad():
        _, masks_pred, _, _ = model(img_tensor, prompt)
    entropy_maps = []
    pred_ins = []
    for i, mask_p in enumerate( masks_pred[0]):

        p = mask_p.clamp(1e-6, 1 - 1e-6)
        if p.ndim == 2:
            p = p.unsqueeze(0)

        entropy_map = entropy_map_calculate(p)
        entropy_maps.append(entropy_map)
        pred_ins.append(p)

    return entropy_maps, pred_ins
        
        
        
    

In [ ]:
import torch
import matplotlib.pyplot as plt

def edge_corner_score(x, y, x_c, y_c, w, h, gamma=1.0):
    dx = 2 * torch.abs(x - x_c) / w
    dy = 2 * torch.abs(y - y_c) / h
    dx = torch.clamp(dx, 0, 1)
    dy = torch.clamp(dy, 0, 1)
    score = (dx + dy - dx * dy) ** gamma
    return score

# Rectangle parameters
w, h = 100, 500
x_c, y_c = w / 2, h / 2

# Create grid of coordinates
x = torch.linspace(0, w, 400)
y = torch.linspace(0, h, 300)
xx, yy = torch.meshgrid(x, y, indexing="xy")

# Compute edge/corner score over the grid
score_map = edge_corner_score(xx, yy, x_c, y_c, w, h, gamma=0.7)

# Plot the heatmap
plt.figure(figsize=(6,4))
plt.imshow(score_map.numpy(), origin="lower", cmap="plasma", extent=[0, w, 0, h])
plt.colorbar(label="Edge/Corner Score")
plt.title("Edge + Corner Proximity Heatmap")
plt.xlabel("x")
plt.ylabel("y")
plt.scatter([x_c], [y_c], c="white", s=30, label="Center")
plt.legend()
plt.show()


In [ ]:

def edge_corner_score(x, y, x_c, y_c, w, h, gamma=0.7):
    dx = 2 * torch.abs(x - x_c) / w
    dy = 2 * torch.abs(y - y_c) / h
    dx = torch.clamp(dx, 0, 1)
    dy = torch.clamp(dy, 0, 1)
    # high on edges + corners, low at center
    score = (dx + dy - dx * dy) ** gamma
    return score


In [ ]:
import torch
import torch.nn.functional as F
import numpy as np

def adjust_bbox_by_gradient(prob_map, bbox, max_iter=50, step=1, gain_thresh=0.001, device='cpu'):
    """
    Expands or shifts a bounding box toward the segmentation region
    guided by gradients in the probability map.
    """
    if isinstance(prob_map, np.ndarray):
        prob_map = torch.from_numpy(prob_map)
    prob_map = prob_map.float().to(device)
    H, W = prob_map.shape

    bbox = torch.tensor(bbox, dtype=torch.float32, device=device)

    # Compute gradients once
    gy, gx = torch.gradient(prob_map)
    grad_mag = torch.sqrt(gx**2 + gy**2)

    def clamp_box(b):
        x1, y1, x2, y2 = b
        return torch.tensor([
            torch.clamp(x1, 0, W-1),
            torch.clamp(y1, 0, H-1),
            torch.clamp(x2, 1, W),
            torch.clamp(y2, 1, H)
        ], device=device)

    for i in range(max_iter):
        x1, y1, x2, y2 = bbox.int()
        x1, y1 = max(0, x1), max(0, y1)
        x2, y2 = min(W-1, x2), min(H-1, y2)
        if x2 <= x1 or y2 <= y1:
            break

        # Measure edge gradient means
        top_grad = grad_mag[y1, x1:x2].mean()
        bottom_grad = grad_mag[y2-1, x1:x2].mean()
        left_grad = grad_mag[y1:y2, x1].mean()
        right_grad = grad_mag[y1:y2, x2-1].mean()

        edges = {
            'up': top_grad,
            'down': bottom_grad,
            'left': left_grad,
            'right': right_grad
        }

        best_edge = max(edges, key=edges.get)
        best_grad = edges[best_edge].item()

        if best_grad < gain_thresh:
            break

        # Move or expand the chosen edge outward
        if best_edge == 'up':
            bbox[1] -= step
        elif best_edge == 'down':
            bbox[3] += step
        elif best_edge == 'left':
            bbox[0] -= step
        elif best_edge == 'right':
            bbox[2] += step

        bbox = clamp_box(bbox)

    # Final mask
    final_mask = torch.zeros_like(prob_map)
    x1, y1, x2, y2 = bbox.int()
    final_mask[y1:y2, x1:x2] = 1.0

    return bbox.cpu().tolist(), final_mask.cpu()


In [ ]:
import os
import torch
import numpy as np
import torchvision
import os
import cv2

save_dir = "entropy_sorted"
os.makedirs(save_dir, exist_ok=True)

W, H = 1024, 1024

for rank, (entropy_scalar, img_path, render) in enumerate(collected, start=1):
    img_name = os.path.splitext(os.path.basename(img_path))[0]

    # ---- Convert and move to device
    img_np = render['real']        # numpy HxWx3
    img_tensor = torch.from_numpy(img_np).permute(2,0,1).float() / 255.0
    img_tensor = img_tensor.unsqueeze(0).to(fabric.device)

    prompt_main = render['prompt']
    # ---- Forward pass to get entropy + predictions
    # entropy_maps, preds = process_forward(img_tensor, prompt_main)

    # print(prompt_main[0][0].shape)
    # pred_stack = torch.stack(preds, dim=0)

    # # Convert to binary masks (e.g., threshold 0.99 as you do)
    # pred_binary = (pred_stack > 0.99).float() 
    # # Count overlaps
    # overlap_count = pred_binary.sum(dim=0)  # (1,1024,1024)
    
    # # Optional: extract overlapping region mask (for debugging)
    # overlap_map = (overlap_count > 1).float()
    # invert_overlap_map = 1.0 - overlap_map
    
    # # Convert to uint8 for visualization/saving
    # # non_overlap_vis = (non_overlap_map[0].cpu().numpy() * 255).astype(np.uint8)
    

    # # # ---- Save the original image once
    img_save_path = os.path.join(save_dir, f"{rank}.jpg")
    cv2.imwrite(img_save_path, cv2.cvtColor(img_np, cv2.COLOR_RGB2BGR))

    # scores = []
    # # ---- Loop over instances (each mask + entropy)
    # for i, (entr_map, pred) in enumerate(zip(entropy_maps, preds)):
    #     # # Normalize entropy 0–1
    #     # entr_norm = (entr_map - entr_map.min()) / (entr_map.max() - entr_map.min() + 1e-8)

    #     # # Convert to uint8
    #     # entr_vis = (entr_norm[0].cpu().numpy() * 255).astype(np.uint8)
    #     pred = (pred[0]>0.99) 
    #     pred_w_overlap = pred * invert_overlap_map[0]

        

        
    #     # Find where mask == 1
    #     ys, xs = torch.where(pred_w_overlap > 0.5)
    #     if len(xs) > 0 and len(ys) > 0:
    #         x_min, x_max = xs.min().item(), xs.max().item()
    #         y_min, y_max = ys.min().item(), ys.max().item()
    #         h, w = y_max - y_min, x_max - x_min
    #         print(f"Bounding box: ({x_min}, {y_min}) → ({x_max}, {y_max})") 
    #         cx = (x_min + x_max) / 2.0
    #         cy = (y_min + y_max) / 2.0
    #         # # Optional: draw rectangle
    #         # mask_vis = (pred_w_overlap.cpu().numpy() * 255).astype(np.uint8)
    #         # mask_rgb = cv2.cvtColor(mask_vis, cv2.COLOR_GRAY2BGR)
    #         # cv2.rectangle(mask_rgb, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
    #         # pred_path = os.path.join(save_dir, f"{rank:04d}_{img_name}_{i}mask_with_box.png")
    #         # cv2.imwrite(pred_path, mask_rgb)
    #     else:
    #         print("No 1s found in mask")
    #     # Make sure both tensors are on same device
    #     point_ref = torch.tensor([cx, cy], dtype=torch.float32, device=prompt_main[0][0][i].device)
    #     score = edge_corner_score(prompt_main[0][0][i][0][0].item(), prompt_main[0][0][i][0][1].item(), point_ref[0], point_ref[1], w, h)
    #     scores.append(score)
    #     # point_ref = torch.tensor([cx, cy], dtype=torch.float32, device=prompt_main[0][0][i].device)
    #     # dist = torch.dist(prompt_main[0][0][i], point_ref)
        
    #     # pred_w_overlap_vis = (pred_w_overlap.cpu().numpy()* 255).astype(np.uint8)
    

    #     # # Save prediction mask
    #     # pred_path = os.path.join(save_dir, f"{rank:04d}_{img_name}_{i}_pred.png")
    #     # cv2.imwrite(pred_path, pred_w_overlap_vis)
    # print(f"Saved image + {len(entropy_maps)} instance maps for {img_name}")


    x_offset = 16
    y_offset = 16
    refine_iter = 20
    for refine in range(refine_iter):

        if refine == 0:
            bboxes = []
            for j in range(len(prompt_main[0][0])):
                x = prompt_main[0][0][j][0][0]
                y = prompt_main[0][0][j][0][1]
    
                bboxes.append(torch.tensor([x-x_offset, y - y_offset, x + x_offset, y + y_offset], dtype=torch.float32))

            bboxes = torch.stack(bboxes)
        with torch.no_grad():
            _, masks_pred, _, _ = model(img_tensor, bboxes.unsqueeze(0))

        # masks_pred[0] shape -> [num_boxes, H, W]
        pred_masks = masks_pred[0].cpu()  
        bbox_new = []
        for i, mask in enumerate(pred_masks):
            # Threshold mask to binary: 0 or 255
            mask_bin = (mask > 0.5).numpy().astype(np.uint8) * 255  # clean mask
            
            
            
            
            
            # -------------------------
            # Compute and save entropy heatmap
            # -------------------------
            # Convert mask to probability (0-1)
            mask_prob = mask.numpy()
            
            # Avoid log(0) by clamping probabilities
            eps = 1e-6
            mask_prob = np.clip(mask_prob, eps, 1 - eps)
            # Compute entropy per pixel
            entropy_map = - (mask_prob * np.log(mask_prob) + (1 - mask_prob) * np.log(1 - mask_prob))
            
            # Normalize entropy to 0-255 for visualization
            entropy_norm = ((entropy_map - entropy_map.min()) / (entropy_map.max() - entropy_map.min()) * 255).astype(np.uint8)
            
            # Crop the entropy map to the bounding box
            entropy_roi = entropy_map[y1:y2, x1:x2]
            
            # Compute mean entropy inside the rectangle
            mean_entropy = entropy_roi.mean()
            
            print(f"Mask {i} mean entropy inside bbox: {mean_entropy:.4f}")
                    

      
            prob_map = mask.numpy()
            opt_box, _ = adjust_bbox_by_gradient(prob_map, bboxes[i].int().tolist())
            opt_box = list(map(int, opt_box))
          
            bbox_new.append(torch.tensor(opt_box, dtype=torch.float32))

            


            

            
            if refine == (refine_iter-1):
                
                # Convert to 3-channel for colored box
                mask_color = cv2.cvtColor(mask_bin, cv2.COLOR_GRAY2BGR)
                
                # Draw bounding box
                x1, y1, x2, y2 = bboxes[i].int().tolist()
                cv2.rectangle(mask_color, (x1, y1), (x2, y2), color=(0, 255, 0), thickness=2)
    
                # Apply a colormap for better visualization
                entropy_color = cv2.applyColorMap(entropy_norm, cv2.COLORMAP_JET)
                # Draw bounding box on entropy map as well
                cv2.rectangle(entropy_color, (x1, y1), (x2, y2), color=(255, 255, 255), thickness=2)
                
                # Save entropy heatmap
                save_path_entropy = os.path.join(save_dir, f"{rank}_{i}_entropy_heatmap.png")
                cv2.imwrite(save_path_entropy, entropy_color)
                print(f"Saved {save_path_entropy}")
    
                # Draw bounding box
                x1, y1, x2, y2 = opt_box
                cv2.rectangle(mask_color, (x1, y1), (x2, y2), color=(0, 255, 255), thickness=2)
                
                
    
                # Save the clean mask with box
                save_path = os.path.join(save_dir, f"{rank}_{i}_mask_clean_box.png")
                cv2.imwrite(save_path, mask_color)
                print(f"Saved {save_path}")
        
        bbox_new = torch.stack(bbox_new)
        bboxes = bbox_new
     

        
    

    # Stop early for testing
    if rank >= 5:
        break
